In [1]:
# Import necessary libraries
import os
from vllm import LLM, SamplingParams
from vllm.steer_vectors.request import SteerVectorRequest, VectorConfig
from transformers import AutoTokenizer

# Set environment variables
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# Initialize LLM with steering vector capability
llm = LLM(
    model="/data/zju-46/shenyl/hf/model/openai-community/gpt2/",
    enable_steer_vector=True,
    enforce_eager=True,
    tensor_parallel_size=1,
    enable_chunked_prefill=False
)


INFO 11-03 14:53:04 [utils.py:253] non-default args: {'disable_log_stats': True, 'enforce_eager': True, 'enable_steer_vector': True, 'enable_chunked_prefill': False, 'model': '/data/zju-46/shenyl/hf/model/openai-community/gpt2/'}
INFO 11-03 14:53:04 [model.py:657] Resolved architecture: GPT2LMHeadModel
ERROR 11-03 14:53:04 [config.py:304] Error retrieving safetensors: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/data/zju-46/shenyl/hf/model/openai-community/gpt2/'. Use `repo_type` argument if needed., retrying 1 of 2
ERROR 11-03 14:53:06 [config.py:302] Error retrieving safetensors: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/data/zju-46/shenyl/hf/model/openai-community/gpt2/'. Use `repo_type` argument if needed.
INFO 11-03 14:53:06 [model.py:1969] Downcasting torch.float32 to torch.bfloat16.
INFO 11-03 14:53:06 [model.py:1746] Using max model len 1024
INFO 11-03 14:53:08 [scheduler.py:211] Chunked prefill is enabled with max_num_batched_to

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


(EngineCore_DP0 pid=747946) INFO 11-03 14:53:11 [default_loader.py:314] Loading weights took 0.47 seconds
(EngineCore_DP0 pid=747946) INFO 11-03 14:53:11 [steer_vector_model_runner_mixin.py:36] Initialized SteerVector worker manager
(EngineCore_DP0 pid=747946) INFO 11-03 14:53:11 [steer_vector_model_runner_mixin.py:50] Wrapping model with steer vector support
(EngineCore_DP0 pid=747946) INFO 11-03 14:53:11 [hidden_states_model_runner_mixin.py:90] Wrapped 12 decoder layers for hidden states capture
(EngineCore_DP0 pid=747946) INFO 11-03 14:53:12 [gpu_model_runner.py:2971] Model loading took 0.2378 GiB and 0.511050 seconds
(EngineCore_DP0 pid=747946) INFO 11-03 14:53:14 [gpu_worker.py:343] Available KV cache memory: 41.91 GiB
(EngineCore_DP0 pid=747946) INFO 11-03 14:53:14 [kv_cache_utils.py:1247] GPU KV cache size: 1,220,720 tokens
(EngineCore_DP0 pid=747946) INFO 11-03 14:53:14 [kv_cache_utils.py:1252] Maximum concurrency for 1,024 tokens per request: 1192.11x
(EngineCore_DP0 pid=74794

In [2]:
# Create a simple example prompt for testing
example = "How to spend our life?"
# Generate baseline response without steering
example_answers = llm.generate(
    example,
    SamplingParams(
        temperature=0,
        max_tokens=50,
        skip_special_tokens=False,
    ),
)

# Display baseline response
print("=====Baseline=====")
print(example_answers[0].outputs[0].text)

sv_request = SteerVectorRequest(
    steer_vector_name="lm_steer",
    steer_vector_int_id=1,
    steer_vector_local_path="/data/zju-48b/xhl/my_lab/github-test/EasySteer/replications/lm_steer/gpt2.pt", # https://huggingface.co/spaces/Glaciohound/LM-Steer/blob/main/checkpoints/gpt2.pt
    prefill_trigger_positions=[-1],
    algorithm="lm_steer",
    scale=1.0, # here positive - Toxification, negative - Detoxification
    target_layers=[11] # last layer
)
# Generate response with lm_steer
output = llm.generate(
    example, 
    SamplingParams(
        temperature=0,
        max_tokens=50,
        skip_special_tokens=False,
    ), 
    steer_vector_request=sv_request
)

# Display lm_steer response
print("=====lm_steer=====")
print(output[0].outputs[0].text)

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%| | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, 

=====Baseline=====


The answer is simple: Spend your life.

The best way to spend your life is to spend it wisely.

The best way to spend your life is to spend it wisely.

The best way to spend your life


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%| | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, 

=====lm_steer=====
ONSORED

The answer is simple: We can't.

The answer is that we can't spend our life.

The answer is that we can't spend our life.

The answer is that we can't spend our life
